In [2]:
from google.colab import files
files.upload()

Saving botdata.json to botdata.json


{'botdata.json': b'{\n  "intents": [\n    {\n      "tag": "Greeting",\n      "patterns": [\n        "Hi",\n        "hy",\n        "hi",\n        "Hey",\n        "How are you",\n        "Is anyone there?",\n        "Hello",\n        "Good day",\n        "How are you?",\n        "Hi there!",\n        "Hey!",\n        "Good morning!",\n        "Good afternoon!",\n        "Good evening!",\n        "Greetings!",\n        "Hi, can you help me?",\n        "Is anyone there?",\n        "Howdy!",\n        "Greetings!",\n        "What\'s up?",\n        "How\'s it going?",\n        "How are you?",\n        "How\'s your day?",\n        "How\'s everything?",\n        "What\'s new?",\n        "Howdy-do?",\n        "Yo!",\n        "Wassup?",\n        "Hi, friend!",\n        "Top of the morning to you!",\n        "Hi, stranger!",\n        "Hi, buddy!",\n        "Hi, mate!"\n        \n      ],\n      "responses": [\n        "Hey ! How can I assist you with your car rental today?",\n        "Hello, thank

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def tokenizede(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stemm(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_wordss(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stemm(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag

In [ ]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# from nltk_utils import bag_of_words, tokenize, stem
# from model import NeuralNet

with open('botdata.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenizede(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stemm(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_wordss(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

# Evaluate the model on the training data
correct = 0
total = 0

with torch.no_grad():
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        outputs = model(words)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Training Accuracy: {accuracy:.2f}%')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


4569 patterns
89 tags: ['Car_Need', 'Default_Intent', 'Goodbye', 'Greeting', 'Small_Talks', 'Thanks', 'additional_services', 'age', 'age_requirement', 'availability', 'cancel_reservation', 'car_accessories', 'car_colors', 'car_dealership_locations', 'car_financing_qualifications', 'car_financing_terms', 'car_fuel_economy', 'car_horsepower_info', 'car_infotainment_systems', 'car_inspection', 'car_insurance', 'car_leasing', 'car_location_finder', 'car_maintenance_schedule', 'car_maintenance_services', 'car_ownership_costs', 'car_price_range', 'car_rental', 'car_reviews', 'car_safety_features', 'car_security', 'car_sizes', 'car_technology_upgrades', 'car_trade_in', 'car_troubleshooting', 'car_type_comparison', 'car_types', 'car_warranty', 'child_seats', 'color', 'condition', 'contact_info', 'corporate_rentals', 'customization', 'delivery', 'discount_code', 'driver_additional_fee', 'driver_requirements', 'electric_cars', 'environmental_impact', 'fallback', 'features', 'financing', 'fuel_ef

In [1]:
import random
import json

import torch
import pandas as pd
# from model import NeuralNet
# from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('botdata.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

admin_data = pd.read_csv('/content/ADMIN DATA - Sheet1.csv')
user_inputs = pd.DataFrame(columns=['user_input', 'pickup_location', 'pickup_datetime', 'dropoff_datetime', 'car_model'])

bot_name = "car_bot"
print("Let's chat! (type 'quit' to exit)")

while True:
    user_input = input("user : ")

    if user_input.lower() in ["quit", "exit"]:
        break
# USER INPUT TOKENIZING
    user_input = tokenizede(user_input)
    X = bag_of_wordss(user_input, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    # PREDICTION
    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    # DECLARE CAR_NEED TAG THAT TRIGGERS USER INPUT FOR CAR AVAILABILITY FEATURE
    if tag == "Car_Need":
        print("car_bot: Sure, I can help you with car rental. Please provide the following details:")
        pickup_location = input("Please enter pickup location: ")
        pickup_datetime = input("Please enter pickup date and time (e.g., 21-09-23 03:00 PM): ")
        dropoff_datetime = input("Please enter drop-off date and time (e.g., 21-09-23 04:00 PM): ")
        car_model = input("Please enter car model: ")

        user_inputs = user_inputs.append({
            'user_input': user_input,
            'pickup_location': pickup_location,
            'pickup_datetime': pickup_datetime,
            'dropoff_datetime': dropoff_datetime,
            'car_model': car_model
        }, ignore_index=True)

        # FILTERING ADMIN DATA
        filtered_data = admin_data[
            (admin_data['location'] == pickup_location) &
            (admin_data['car_model'] == car_model)
            (admin_data['availability'] == "Available") # avilability status check
        ]

        if len(filtered_data) > 0:
            print("car_bot: Car is available.")
            for index, row in filtered_data.iterrows():
                print(f"Price for {row['car_model']} in {row['location']}: ${row['price']:.2f}")
            # UPDATE ADMIN DATA AVAILABILITY COLUMN FROM 1 TO 0
            admin_data.loc[filtered_data.index, 'availability'] = "Not Available"
            admin_data.to_csv('/content/ADMIN DATA - Sheet1.csv', index=False)  # Save updated admin_data to CSV
        else:
            # Car is not available
            print("car_bot: Car is not available. Please choose another car.")
    else:
        response_found = False  # Track if a valid response is found
        for intent in intents['intents']:
            if tag == intent["tag"]:
                responses = intent['responses']
                response = random.choice(responses)
                print(f"car_bot: {response}")
                response_found = True
                break  # Break the loop after printing one response

        if not response_found:
            print("car_bot: I do not understand...")
    # print()  # Add a line break for readability

user_inputs.to_csv('user_inputs.csv', index=False)
print("Goodbye! If you have more questions, feel free to ask.")


FileNotFoundError: ignored